In [1]:
import pandas as pd
import numpy as np
import gensim

/Users/Sym/anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [4]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

path_emails = 'hillary-clinton-emails/Emails.csv'

emails = pd.read_csv(path_emails, usecols = ['ExtractedSubject', 'ExtractedBodyText', 'SenderPersonId'])
emails.fillna(' ', inplace = True)
emails['subject_body'] = emails['ExtractedSubject'] + ' ' + emails['ExtractedBodyText']
emails_raw = np.asarray(emails['subject_body'])

email_tokens = []

for email in emails_raw:
    email_tokens.append(regexp_tokenize(email, pattern='\w+'))

stop_words = set(stopwords.words('english'))
stopwords_emails = ['fm', 'am', 'pm', 'n\'t', 'sent', 'from', 'to', 'subject', 'fw', 'fwd', 'fvv', 'cc', 'bcc', 'attachments', 're', 'date', 'html', 'php']
stop_words.update(stopwords_emails)

email_clean_tokens = []

# remove stopwords and numbers
for email in email_tokens:
    clean_tokens = [token for token in email if token.lower() not in stop_words and token.isdigit() == False]
    email_clean_tokens.append(clean_tokens)

stemmer = PorterStemmer()

email_clean = []

for email in email_clean_tokens:
    clean = [stemmer.stem(token.lower()) for token in email]
    email_clean.append(clean)

SenderPersonId        157
ExtractedSubject     1685
ExtractedBodyText    1203
dtype: int64


In [7]:
dictionary = gensim.corpora.Dictionary(email_clean)
corpus = [dictionary.doc2bow(email) for email in email_clean]

In [8]:
number_topics = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

for n in number_topics:
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n, id2word=dictionary, passes=10)
   
    print('Model with', n, 'topics\n')
   
    topic_id = 1
   
    for topic in lda_model.show_topics(num_topics=n, num_words=10, log=False, formatted=False):
        print('topic #', topic_id)
        print(topic)
        #for prob, word_id in topic:
            #print(dictionary[word_id])
        #topic_id += 1

LdaModel(num_terms=19547, num_topics=50, decay=0.5, chunksize=2000)
